Test the RLBench input output functionality for the internal tasks

In [1]:
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import EndEffectorPoseViaPlanning
from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.observation_config import ObservationConfig
from rlbench.tasks import FS10_V1, ReachTarget, PickUpCup, PutGroceriesInCupboard, StackBlocks, PickAndLift, OpenDrawer, CloseDrawer, HangFrameOnHanger, EmptyDishwasher, BasketballInHoop
import torch
import numpy as np


Launch the RLBench environment with all the observations enabled

In [2]:
obs_config = ObservationConfig()
obs_config.set_all(True)

env = Environment(
    action_mode=MoveArmThenGripper(
        arm_action_mode=EndEffectorPoseViaPlanning(absolute_mode=True), gripper_action_mode=Discrete()),
    obs_config=ObservationConfig(),
    headless=False)
env.launch()

In [3]:
import pprint as pp

# Get the task
task = env.get_task(ReachTarget)
pp.pprint(task.variation_count())
demo = task.get_demos(1, live_demos=True, random_selection=False)[0]



20


In [4]:
# Reset the task
descriptions, obs = task.reset_to_demo(demo)
instruction = descriptions[1] # Could make this random at some point
pp.pprint(instruction)

'touch the red ball with the panda gripper'


In [ ]:
# from data.utils import keypoint_discovery
# pp.pprint(type(demo))
# pp.pprint(len(demo))
# keypoints = keypoint_discovery(demo)
# pp.pprint(keypoints)
# obs = demo[keypoints[0]]
# type(obs)

In [5]:
from data.utils import keypoint_discovery
keypoints = keypoint_discovery(demo)

for i, keypoint in enumerate(keypoints):
    obs = demo[keypoint]
    action = np.concatenate([obs.gripper_pose, [obs.gripper_open]])
    pp.pprint(action)
    obs, reward, terminate = task.step(action)
    print(f'Step: {i} Reward: {reward}')

array([ 4.20428336e-01, -1.77622497e-01,  7.97428846e-01,  2.23513280e-06,
        1.00000000e+00,  1.28546322e-04, -1.44612830e-04,  1.00000000e+00])
Step: 0 Reward: 1.0


In [6]:
env.shutdown()

[CoppeliaSim:loadinfo]   done.
